In [1]:
import pandas as pd

In [2]:
#Read csv file 
data = pd.read_csv('bbc-news-data.csv', sep="\t")

In [3]:
data.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [4]:
import nltk

In [5]:
nltk.download('stopwords') # Download stopwords from Natural Language Toolkit
nltk.download('punkt')     #Download punctuation from Natural Language Toolkit 

[nltk_data] Downloading package stopwords to /home/nahid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/nahid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk_stopwords = nltk.corpus.stopwords.words('english')    #Choose English language from Stopwords
len(nltk_stopwords)

179

In [7]:
from nltk.stem.snowball import SnowballStemmer    #Using Stemming algorithm for root word

In [8]:
sno = SnowballStemmer('english')    #Provide language as prameter 

In [9]:
sno.stem('grows')

'grow'

In [10]:
from nltk import word_tokenize

In [11]:
dataset = pd.DataFrame(columns = ('title_body', 'category')) #Create Empty dataframe with two columns

#iterate each row of dataset
for index, row in data.iterrows():
    title_body = row['title'] + ' ' + row['content']
    title_body_tokenized = word_tokenize(title_body)  #Convert string to list 
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in nltk_stopwords] #Filtered list by remove stopwords from the list
    title_body_tokenized_filtered_snoStemmed = [sno.stem(w) for w in title_body_tokenized_filtered]  #Use Stemming algorithm to stem filtered list
    dataset.loc[index] = {    #Fill up columns of dataset
        'title_body': ' '.join(title_body_tokenized_filtered_snoStemmed), 
        'category': row['category']
    }

In [12]:
dataset.head()

,title_body,category
0,ad sale boost time warner profit quarter profi...,business
1,dollar gain greenspan speech the dollar hit hi...,business
2,yuko unit buyer face loan claim the owner emba...,business
3,high fuel price hit ba 's profit british airwa...,business
4,pernod takeov talk lift domecq share uk drink ...,business


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer()

In [15]:
vectorizer.fit(dataset['title_body'])

TfidfVectorizer()

In [16]:
X = vectorizer.transform(dataset['title_body'])

In [17]:
X

<2225x20904 sparse matrix of type '<class 'numpy.float64'>'
	with 346158 stored elements in Compressed Sparse Row format>

In [18]:
#We need to convert dependent variable to label(numbers)
from sklearn.preprocessing import LabelEncoder

In [19]:
le = LabelEncoder()

In [20]:
y = le.fit_transform(dataset['category'])

In [21]:
len(y)

2225

In [22]:
import numpy as np
print(np.unique(dataset['category']))

['business' 'entertainment' 'politics' 'sport' 'tech']


In [23]:
#Display the number of row and columns
np.shape(X)

(2225, 20904)

In [24]:
#Display the number of row
np.shape(y)

(2225,)

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y) 

In [27]:
from sklearn import svm

In [28]:
svmc = svm.SVC()
svmc.fit(X_train, y_train)

SVC()

In [29]:
#Display the accuracy 
svmc.score(X_test, y_test)  #Predict X_test and compare it with y_test

0.9748653500897666

In [30]:
from sklearn.metrics import classification_report, confusion_matrix

In [31]:
y_pred = svmc.predict(X_test)

In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       147
           1       1.00      0.98      0.99        93
           2       0.94      0.96      0.95       100
           3       1.00      1.00      1.00       132
           4       0.97      0.98      0.97        85

    accuracy                           0.97       557
   macro avg       0.97      0.97      0.97       557
weighted avg       0.98      0.97      0.97       557



In [33]:
print(confusion_matrix(y_test, y_pred))

[[141   0   5   0   1]
 [  1  91   1   0   0]
 [  2   0  96   0   2]
 [  0   0   0 132   0]
 [  2   0   0   0  83]]
